In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
print(f"Đã import các thư viện cần thiết")

Đã import các thư viện cần thiết


# định nghĩa hàm tạo model transfer learning

In [2]:
def get_pretrained_resnet(num_classes=1,freeze_backbone=True):
    print(f"Bắt đầu tải model")
    weights=models.ResNet50_Weights.IMAGENET1K_V2 
    model=models.resnet50(weights=weights)
    print(f"Đã tải xong model")

    #đóng băng trọng số backbone
    if freeze_backbone:
        print(f"Đang đóng băng trọng số backbone")
        for name,param in model.named_parameters():
            if 'fc' not in name:
                param.requires_grad=False
        print(f"Đã đóng băng trọng số backbone")
    else:
        print(f"Không đóng băng trọng số backbone")
    
    #thay thế lớp phân loại cuối cùng
    print(f"Đang thay thế lớp phân loại cuối cùng")
    num_features=model.fc.in_features
    print(f"Số lượng đặc trưng đầu vào của lớp phân loại cuối cùng: {num_features}")
    #tạo đối tượng lớp phân loại mới
    new_fc_layer=nn.Linear(in_features=num_features,out_features=num_classes)
    #in thông tin về lớp phân loại mới
    print(f"Lớp phân loại mới: {new_fc_layer}")
    #thay thế lớp phân loại cuối cùng
    model.fc=new_fc_layer
    print(f"Đã thay thế lớp phân loại cuối cùng")

    #kiểm tra tham số của model cần huấn luyện 
    print(f"Thông tin về các tham số của model")
    trainable_param_count=0
    for name,param in model.named_parameters(): #lặp qua các tham số của model
        if param.requires_grad: #chỉ xét tham số có requires_grad=True
            print(f"Tên tham số: {name}, kích thước: {param.size()}, requires_grad: {param.requires_grad}")
            trainable_param_count+=param.numel()
    print(f"Tổng số tham số có thể huấn luyện: {trainable_param_count}")
    return model

# Phần kiểm tra

In [3]:
if __name__=='__main__':
    print(f"Test model")
    #test 1: tạo model với backbone bị đóng băng
    print(f"Test 1: Tạo model với backbone bị đóng băng")
    test_model_frozen=get_pretrained_resnet(num_classes=1,freeze_backbone=True)
    print(f"In ra lớp cuối cùng của model: {test_model_frozen.fc}")

    #test 2: tạo model với backbone không bị đóng băng
    print(f"Test 2: Tạo model với backbone không bị đóng băng")
    test_model_unfrozen=get_pretrained_resnet(num_classes=1,freeze_backbone=False)
    print(f"In ra lớp cuối cùng của model: {test_model_unfrozen.fc}")
    print("Layer1[0].conv1.weight requires_grad:", test_model_unfrozen.layer1[0].conv1.weight.requires_grad) # Mong đợi True.
    
    #test 3: kiểm tra forrward pass có chạy được không
    print(f"Kiểm tra forward pass có chạy được không")
    dummy_input=torch.randn(4,3,244,244) #tạo đầu vào giả với kích thước (batch_size, num_channels, height, width)
    try: #bắt lỗi nếu có
        output=test_model_frozen(dummy_input)
        print(f"Đầu ra của model: {output}")
        print(f"Input có dạng {dummy_input.shape}")
        print(f"Output có dạng {output.shape}")
    except Exception as e:
        print(f"Đã xảy ra lỗi: {e}")
    print(f"Đã kiểm tra xong forward pass")

Test model
Test 1: Tạo model với backbone bị đóng băng
Bắt đầu tải model
Đã tải xong model
Đang đóng băng trọng số backbone
Đã đóng băng trọng số backbone
Đang thay thế lớp phân loại cuối cùng
Số lượng đặc trưng đầu vào của lớp phân loại cuối cùng: 2048
Lớp phân loại mới: Linear(in_features=2048, out_features=1, bias=True)
Đã thay thế lớp phân loại cuối cùng
Thông tin về các tham số của model
Tên tham số: fc.weight, kích thước: torch.Size([1, 2048]), requires_grad: True
Tên tham số: fc.bias, kích thước: torch.Size([1]), requires_grad: True
Tổng số tham số có thể huấn luyện: 2049
In ra lớp cuối cùng của model: Linear(in_features=2048, out_features=1, bias=True)
Test 2: Tạo model với backbone không bị đóng băng
Bắt đầu tải model
Đã tải xong model
Không đóng băng trọng số backbone
Đang thay thế lớp phân loại cuối cùng
Số lượng đặc trưng đầu vào của lớp phân loại cuối cùng: 2048
Lớp phân loại mới: Linear(in_features=2048, out_features=1, bias=True)
Đã thay thế lớp phân loại cuối cùng
Thông